This notebook performs the preprocessing of the raw data.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path

import pandas as pd
import numpy as np

from src.preprocess import parse_all_data, clean_raw_data, computeTimeDeltaTraces, computePowerDeltaTraces

import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "none"

import src.colors as colors
from src.helpers import Modes, Parameters
from src.stats import ThompsonCI_twosided
from src.plots import prr_f_TimeDelta, prr_f_PowerDelta, prr_3d, prr_matrix_plot

In [15]:
# Set to False to view larger versions of the plots
# Set to True  to reproduce the exact plots from the paper
PaperPlot = True

# Select the different output format settings
if PaperPlot:
    output_format = 'paper'
else:
    output_format = 'online'

if output_format == 'online':
    font_size_px = 14
    linewidth_px = 512
    landscapewidth_px = 654
    plot_path = None

if output_format == 'paper':
    font_size_pt = 6
    offset = 5 # to compensate for the rounding of unit conversions
    linewidth_pt = 241 - offset  
    landscapewidth_pt = 506 - offset
    
    # 1pt = 1.333px
    font_size_px = int(font_size_pt*1.333)+1
    linewidth_px = int(linewidth_pt*1.333)+1
    landscapewidth_px = int(landscapewidth_pt*1.333)+1

    plot_path = Path('plots_cps-iotbench')
    
# Create plot directory if don't exist
if (plot_path is not None) and (not os.path.exists(plot_path)):
    os.mkdir(plot_path)

In [4]:
# Clean up and bug correction of the data collected by Anna
# clean_raw_data()

In [5]:
# Load data
df = parse_all_data()
DataPath = Path('data_preprocessed')
# display(df)

Processed data retrieved.


In [124]:
plot_name = 'prr_matrix.pdf'

# Adjust the plot size    
if output_format == 'paper':
    custom_layout = dict(
        font = {"size":font_size_px},
        margin = dict(l=100, r=10, t=30, b=40),
        width = landscapewidth_px
    )
else: 
    custom_layout = None
    
figure = prr_matrix_plot(
    df,
    DataPath,
    PowerDeltaList = [0,2,4,6,8,10],
    layout = custom_layout,
    SamePayload=1,
    showMarkers=False,
    )
figure.update_layout(custom_layout)
figure.show()
figure.write_image(str(plot_path / plot_name))

In [121]:
plot_name = 'prr_matrix_DiffPayload.pdf'
figure = prr_matrix_plot(
    df,
    DataPath,
    PowerDeltaList = [0,2,4,6,8,9],
    layout = custom_layout,
    SamePayload=0
    )
figure.update_layout(custom_layout)
figure.show()
figure.write_image(str(plot_path / plot_name))

In [119]:

ModeList = [i for i in Modes]

# Adjust the plot size    
if output_format == 'paper':
    custom_layout = dict(
        font = {"size":font_size_px},
        margin = dict(l=30, r=5, t=25, b=25),
        width = 0.5*landscapewidth_px,
        height = 150,
        title = '',
        showlegend = False
    )
else: 
    custom_layout = None
    
# for i in [ModeList[0]]:
for i in Modes:
    
    # With same payload
    plot_name = 'CIdelayBound_%s.pdf' % i
    figure = prr_f_TimeDelta(
        df,
        PowerDelta=0,
        SamePayload=1,
        TransPair='all',
        ModesToShow=[i],
        DataPath=DataPath,
        showMarkers=True,
        showCI=True,
        showTimeThreshold=True
        )
    figure.update_layout(custom_layout)
    figure.show()
    figure.write_image(str(plot_path / plot_name))
    
    # With different payload
    plot_name = 'CIdelayBound_%s_DiffPayload.pdf' % i
    figure = prr_f_TimeDelta(
        df,
        PowerDelta=0,
        SamePayload=0,
        TransPair='all',
        ModesToShow=[i],
        DataPath=DataPath,
        showMarkers=True,
        showCI=True,
        showTimeThreshold=True
        )
    figure.update_layout(custom_layout)
    figure.show()
    figure.write_image(str(plot_path / plot_name))

In [102]:
ModeList = [i for i in Modes]

# Adjust the plot size    
if output_format == 'paper':
    custom_layout = dict(
        font = {"size":font_size_px},
        margin = dict(l=30, r=5, t=0, b=25),
        width = 0.4*landscapewidth_px,
        height = 150,
        title = '',
        showlegend = False,
        xaxis=dict(range=[-10,10])
    )
else: 
    custom_layout = None
        
# With same payload
plot_name = 'Power_SamePayload.pdf'
figure = prr_f_PowerDelta(
    df,
    TimeDelta=0,
    SamePayload=1,
    TransPair='all',
    ModesToShow=ModeList,
    DataPath=DataPath,
    showMarkers=False,
    showCI=True,
    )
figure.update_layout(custom_layout)
figure.show()
figure.write_image(str(plot_path / plot_name))

# With different payload
plot_name = 'Power_DiffPayload.pdf'
figure = prr_f_PowerDelta(
    df,
    TimeDelta=0,
    SamePayload=0,
    TransPair='all',
    ModesToShow=ModeList,
    DataPath=DataPath,
    showMarkers=False,
    showCI=True,
    )
figure.update_layout(custom_layout)
figure.show()
figure.write_image(str(plot_path / plot_name))

# "Legend only" plot
plot_name = 'legend.pdf'
figure = go.Figure()
for mode in Modes:
    figure.add_trace(
        dict(
            x=[np.nan],
            y=[np.nan],
            marker={'color':Modes[mode]['color']},
            name=Modes[mode]['label'],
            hoverinfo='skip',
        )
    )
layout = dict(
    font = {"size":font_size_px},
    margin = dict(l=0, r=0, t=0, b=0,),
    width = 0.12*landscapewidth_px,
    height = 100,
    legend = dict(
        orientation = 'v',
        xanchor = 'center',
        yanchor = 'middle',
        x = 0.5,
        y = 0.5
        )
    )
figure.update_layout(layout)
figure.show()
figure.write_image(str(plot_path / plot_name))

In [11]:
filter = (
    (df['SamePayload'] == 0) &
    (df["TimeDelta"] == 0)
)
# filter

filter = filter & (df['TransPair'] == 'A')
filtered_df = df.where(filter).dropna()
filtered_df


# Create and save the median values for faster plotting of the different settings
# computeTimeDeltaTraces(all_pairs=True)

# print(sorted(df['PowerDelta'].unique()))
# print(df['PowerDelta'].unique())
# computePowerDeltaTraces(all_pairs=True)

Processed data retrieved.


,LocalExpCount,TxPowerA,TxPowerB,Mode,TimeDelta,RxCount,TxCount,RssiA,RssiB,PRR,PowerDelta,DateTime,SamePayload,TransPair
GlobalExpCount,,,,,,,,,,,,,,
206,206.0,8.0,2.0,0.0,0.0,20.0,20.0,-60.0,-95.0,100.0,-35.0,2019-09-06 11:16:09,0.0,A
265,265.0,8.0,3.0,0.0,0.0,20.0,20.0,-60.0,-94.0,100.0,-34.0,2019-09-06 11:16:09,0.0,A
324,324.0,8.0,4.0,0.0,0.0,20.0,20.0,-60.0,-94.0,100.0,-34.0,2019-09-06 11:16:09,0.0,A
383,383.0,8.0,5.0,0.0,0.0,20.0,20.0,-60.0,-93.0,100.0,-33.0,2019-09-06 11:16:09,0.0,A
442,442.0,8.0,6.0,0.0,0.0,20.0,20.0,-60.0,-93.0,100.0,-33.0,2019-09-06 11:16:09,0.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27834,5634.0,4.0,8.0,4.0,0.0,20.0,20.0,-61.0,-69.0,100.0,-8.0,2019-09-06 13:49:21,0.0,A
27893,5693.0,5.0,8.0,4.0,0.0,20.0,20.0,-61.0,-69.0,100.0,-8.0,2019-09-06 13:49:21,0.0,A
27952,5752.0,6.0,8.0,4.0,0.0,20.0,20.0,-60.0,-69.0,100.0,-9.0,2019-09-06 13:49:21,0.0,A


In [6]:
# pd.read_csv('data_preprocessed/transmitter_pair_a/different_data/PowerDeltaTraces_A_0_(-140).csv')

In [11]:
import dash-core-components as asd
print(asd.__version__)

SyntaxError: invalid syntax (<ipython-input-11-2e9fa05952cd>, line 1)

In [7]:
n_samples = 10
percentile = 50
confidence = 75
ThompsonCI_twosided(n_samples, percentile, confidence)

(2, 7)

In [17]:
# Helix equation
t = np.linspace(0, 10, 50)
x, y, z = np.cos(t), np.sin(t), t
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                   mode='markers')])
print(go.Scatter3d)

print("\n\nJSON Representation of A Graph Object:\n" + str(fig.to_json()))


<class 'plotly.graph_objs.Scatter3d'>


JSON Representation of A Graph Object:
{"data":[{"mode":"markers","type":"scatter3d","x":[1.0,0.9792475210564969,0.9178514149905888,0.8183599245989672,0.6849024400004521,0.5230181084730104,0.33942593237925484,0.14174589725634046,-0.061817295362854206,-0.262814763741325,-0.4529041164186286,-0.6241957028171256,-0.7695800728569471,-0.883023054382162,-0.95981620122199,-0.996772217050833,-0.9923572439880433,-0.9467545253046645,-0.8618567999191831,-0.7411877443484259,-0.5897557226621226,-0.41384591454310704,-0.22075944916926946,-0.018510372154503518,0.18450697707700792,0.37986637199507933,0.5594594291408053,0.7158321462405541,0.842494280256423,0.9341887246502055,0.9871097053688656,0.9990607393363355,0.9695457993910898,0.8997899018725932,0.7926882623697213,0.6526861299196696,0.48559428713386943,0.29834787391040735,0.09871854474461515,-0.10500809346346798,-0.30437637517455507,-0.4911115282522227,-0.6574631180319587,-0.7965267287855186,-0.9025305312049614

In [36]:
# Compute (or retrieve) all preprocessed data
df = parse_all_data(force_computation='')

# Parameters
PowerDelta = 5
TimeDelta = -140
SamePayload = 0
TransPair = "A"
ModesToShow = [mode for mode in Modes]
# ModesToShow = [ModesToShow[1]]

# print(ModesToShow)

# prr_f_PowerDelta(
#     df,
#     TimeDelta,
#     SamePayload,
#     TransPair,
#     ModesToShow)

fig = prr_3d(
    df,
    SamePayload,
    TransPair,
    ModesToShow)


threeD_plot = go.Figure()
threeD_plot=fig
threeD_plot.show()

Processed data retrieved.


Processed data retrieved.


In [32]:
# Compute (or retrieve) all preprocessed data
df = parse_all_data()
data_path = Path('data_preprocessed')

PowerDelta,SamePayload,TransPair = 9, 1, "B"

filter = (
#     (df['DateTime'] == "2019-10-04 13:55:00") & 
    (df['TransPair'] == "B") & 
    (df['SamePayload'] == 1) & 
    (df["PowerDelta"] == 9)  
)
filtered_df = df.where(filter).dropna()
# display(filtered_df)

Processed data retrieved.


In [60]:
figure = go.Figure()


# Load the median and CI data
file_path = data_path / Parameters['TransPair'][TransPair]['path'] / Parameters['SamePayload'][SamePayload]['path']
file_name = 'TimeDeltaTraces_%s_%s_(%i).csv' % (TransPair,SamePayload,PowerDelta)
df_median = pd.read_csv(file_path / file_name)

for mode in Modes:
    
    mode_filter = (filtered_df["Mode"] == Modes[mode]['id'])
    mode_df = filtered_df.where(mode_filter).dropna()
    
    # Force displaying the trace, even if empty
    if len(mode_df) == 0:
        x_data = [np.nan]
        y_data = [np.nan]
        x_median = [np.nan]
        y_median = [np.nan]
    else: 
        x_data = mode_df["TimeDelta"].to_list()
        y_data = mode_df["PRR"].to_list()
        
        # Extract median data
        x_median = df_median["TimeDelta"].to_list()
        y_median = df_median['median_'+mode].to_list()

        # Extract CI data
        y_LB = df_median['LB_'+mode].to_list()
        y_UB = df_median['UB_'+mode].to_list()
        
        # Prepare the trace for the CI area
        x_CI = x_median + x_median[::-1]
        y_CI = y_LB + y_UB[::-1]
    
    scatter = go.Scatter(
        x=x_data,
        y=y_data,
        mode='markers',
        marker={'color':Modes[mode]['color']},
        showlegend=True,
        legendgroup=Modes[mode]['id'],
        name=Modes[mode]['label'],
        opacity=0.7,
        hoverinfo='skip',
    )
    figure.add_trace(scatter)
    
    
    median_line = go.Scatter(
        x=x_median,
        y=y_median,
        mode='lines',
        line={'color':Modes[mode]['color']},
        showlegend=False,
        legendgroup=Modes[mode]['id'],
        name=Modes[mode]['label']+' median',
    )
    figure.add_trace(median_line)
    
    CI = go.Scatter(
        x=x_CI,
        y=y_CI,
        mode='lines',
        line={
            'color':Modes[mode]['color'],
            'width':0
        },
        showlegend=False,
        legendgroup=Modes[mode]['id'],
        name=Modes[mode]['label']+' CI',
        fill='toself',
        hoverinfo='skip',
        opacity=0.3,
    )
    figure.add_trace(CI)
    
    
    
custom_layout = {
    "title":{
        'text':'PRR = f(Time Delta) <br> with Power Delta = %d dB' % 0,
    },
#     "width":linewidth_px,
#     "height":220,
#     "font":{"size":font_size},
    "margin":dict(t=150),
    "xaxis":{
        'title':{'text':'Transmitters Time Delta [ticks]'},
            },
    "yaxis":{
        'range':[0,103]
    },
    "legend":{
#         'font':{'size':font_size},
        'orientation':"h",
        'x': 0.5,
        'y':1.15,
        'xanchor':"center",
        'yanchor':"top",
    }
}
figure.update_layout(custom_layout)
figure.show()